# Preprocess

In [1]:
from bcd.container import BCDContainer
from bcd.preprocess.convert import ImageConverter, ImageConverterParams
from bcd.preprocess.filter import MeanFilter, MedianFilter, GaussianFilter, FilterParams


In [2]:
container = BCDContainer()
container.init_resources()
container.wire(
    packages=[
        "bcd.manage_data",
        "bcd.preprocess.convert",        
    ]
)

In [3]:
image_repo = container.repo.image()


[10/24/2023 04:01:30 PM] [INFO] [MySQLDatabase] [connect] : Database is not started. Starting database...
[sudo] password for john: 

\nStarting MySQL...
 * Starting MySQL database server mysqld


su: warning: cannot change directory to /nonexistent: No such file or directory


   ...done.


In [4]:
def visualize(processor: str, n: int) -> None:
    condition = lambda df: df['processor'] == processor
    images = image_repo.get(condition=condition, n=n)
    for image in images:
        image.visualize()


## Stage 0: Convert Images

In [5]:
params = ImageConverterParams()
conv = ImageConverter(params=params, task_id="some task id")
conv.execute()
df = image_repo.get_meta()
# visualize(task='ImageConverter', n=4)
df.head()

  0%|          | 0/36 [00:00<?, ?it/s]


AttributeError: 'ImageFactory' object has no attribute '_get_stage'

## Stage 1: Filter

### Stage 1.0: Mean Filter

In [ ]:
params = FilterParams()
filter = MeanFilter(params)
filter.execute()
visualize(task='MeanFilter', n=4)

### Stage 1.1: Median Filter

In [ ]:
params = FilterParams()
filter = MedianFilter(params)
filter.execute()
visualize(task='MedianFilter', n=4)

### Stage 1.1: Gaussian Filter

In [ ]:
params = FilterParams()
filter = GaussianFilter(params)
filter.execute()
visualize(task='GaussianFilter', n=4)

In [7]:
import uuid
str(uuid.uuid4())

'1fc60da3-2bcd-44dd-97b5-0cc6caa33f3e'